In [84]:
import pandas as pd
from collections import deque, defaultdict
import heapq
import time
from concurrent.futures import ProcessPoolExecutor, as_completed

choice_path = '/home/it/Desktop/RUN/NEUT_Official/NEUT_R1/Codes/AN_Virtualized_Choices.csv'  # Update with your actual file path
seats_path = '/home/it/Desktop/RUN/NEUT_Official/NEUT_R1/OfficialData/R1_App_Seats.csv'
rank_path = '/home/it/Desktop/RUN/NEUT_Official/NEUT_R1/OfficialData/R1_App_RankDetail.csv'
candidate_profile_path = '/home/it/Desktop/RUN/NEUT_Official/NEUT_R1/OfficialData/R1_App_CandidateProfile.csv'
quali_marks_path = '/home/it/Desktop/RUN/NEUT_Official/NEUT_R1/OfficialData/R1_App_QualificationMarksDetail.csv'

ip_seats_path = '/home/it/Desktop/RUN/NEUT_Official/NEUT_R1/Data/NEUT_CSAB_NEUT_AN/CSAB_NITK_AndamanIPSeat_new.csv'
p_seats_path = '/home/it/Desktop/RUN/NEUT_Official/NEUT_R1/Data/NEUT_CSAB_NEUT_AN/CSAB_NITK_AndamanPSeat_new.csv'
allotment_path = "/home/it/Desktop/RUN/NEUT_Official/NEUT_R2/NEUT_R2_stronger/NEUT24_R2_App_Allotment.csv"

In [85]:
class Program:
    def __init__(self, boardId,streamId,quotaId,instituteId,programId,categoryId,subcategoryId,genderId,
        tSeat):
        self.board_id = boardId
        self.stream = streamId
        self.quota = quotaId
        self.institute = instituteId
        self.program = programId
        self.category = categoryId
        self.subcategory = subcategoryId
        self.gender = genderId
        self.capacity = tSeat
        self.waitlist = []

    def add_candidate(self, rk, roll_no):
        if rk and rk>0:
            heapq.heappush(self.waitlist, (-rk, roll_no))
            return True
        return False

    def remove_least_preferred(self):
        return heapq.heappop(self.waitlist)

# Function to load candidates from the CSV file
def load_iprograms(filename):
    df = pd.read_csv(filename)
    return {(row['instituteId'], row['programId'], '01', 0, 'NO', 'B'): Program(0,
        row['streamId'],
        '01',
        row['instituteId'],
        row['programId'],
        0,
        'NO',
        'B',
        row['tseat']) for index, row in df.iterrows()}    

# Load candidates from the CandidateProfile.csv file
iprograms = load_iprograms(ip_seats_path)

def load_programs(filename):
    df = pd.read_csv(filename)
    return {(0, row['programId'], '01', row['categoryId'], row['subcategoryId'], 'B'): Program(0,
        row['streamId'],
        '01',
        0,
        row['programId'],
        row['categoryId'],
        row['subcategoryId'],
        'B',
        row['tseat']) for index, row in df.iterrows()}

# Load candidates from the CandidateProfile.csv file
programs = load_programs(p_seats_path)


In [86]:
# Define the Candidate class
class Candidate:
    def __init__(self, board_id, roll_no, state_id, domicile_id, gender_id, 
                 category_id, category_certificate_no):
        self.board_id = board_id
        self.roll_no = roll_no
        self.state_id = state_id
        self.domicile_id = domicile_id
        self.gender_id = gender_id
        self.category_id = category_id
        self.category_certificate_no = category_certificate_no
        self.choice_list = {}
        self.choice_list_mapper = {}
        self.current_index = 1
        self.max_index = 0
        self.rank01 = None
        self.rank03 = None
        self.boardRank = None
        self.cAlloc = None

# Function to load candidates from the CSV file
def load_candidates(filename):
    df = pd.read_csv(filename)
    df = df[(df['domicileId']==1) | (df['domicileId']=='1') | (df['domicileId']=='01')]
    return {row['rollno']: Candidate(
        row['boardId'],
        row['rollno'],
        row['stateId'],
        row['domicileId'],
        'B',
        row['categoryId'],
        row.get('categoryCertificateNo', None)) for index, row in df.iterrows()}

# Load candidates from the CandidateProfile.csv file
candidates = load_candidates(candidate_profile_path)

In [87]:
def choiceReader(candidates, ch_df):
    for index, row in ch_df.iterrows():
        roll_no = int(row['rollno'])
        choice_no = row['ChoiceNumber']
        v_choice_no = int(row['ChoiceNumber_V'])
        inst_code = row['program_code']
        brcode = row['program_name']
        cat = row['Cat_V']
        subcat = str(row['SubCat_V']).upper()
        if roll_no in candidates.keys():
            program_key = (inst_code, brcode, '01', cat, subcat, 'B')
            if (0, brcode, '01', cat, subcat, 'B') not in programs.keys():
                continue
            loaded_prog = programs[0, brcode, '01', cat, subcat, 'B']
            if program_key not in actPrograms.keys():
                actPrograms[inst_code, brcode, '01', cat, subcat, 'B']=Program(loaded_prog.board_id,loaded_prog.stream,'01',inst_code,brcode,cat,subcat,'B',0)
            # program = actPrograms.get(program_key)
            program = actPrograms[inst_code, brcode, '01', cat, subcat, 'B']
            candidates[roll_no].choice_list[candidates[roll_no].max_index] = program
            candidates[roll_no].choice_list_mapper[candidates[roll_no].max_index] = choice_no
            candidates[roll_no].max_index += 1
    return candidates

actPrograms = {}
ch_df = pd.read_csv(choice_path)
candidates = choiceReader(candidates, ch_df)            

In [88]:
def create_candidates_df(candidates):
    # List to store roll numbers with at least one choice
    roll_numbers = []

    # Iterate through candidates to find those with at least one choice
    for roll_no, candidate in candidates.items():
        if candidate.choice_list:  # Check if choice_list is not empty
            roll_numbers.append(roll_no)

    # Create DataFrame from the list of roll numbers
    candidates_df = pd.DataFrame(roll_numbers, columns=['rollNo'])
    
    return candidates_df

candidates_df = create_candidates_df(candidates)

In [ ]:
df = pd.read_csv(rank_path)

info_df = pd.read_csv(candidate_profile_path)
info_df = info_df[(info_df['domicileId']==1) | (info_df['domicileId']=='1') | (info_df['domicileId']=='01')]
info_df = info_df.rename(columns={'rollno': 'rollNo'})
def assign_dom_priority(row):
    if row['categoryId'] in [2, '02']:
        return 2
    else:
        return 1

merged_df = pd.merge(df, info_df, on='rollNo')
print(merged_df.columns)
merged_df1 = merged_df[(merged_df['streamId'] == '01') | (merged_df['streamId'] == 1)]
merged_df1['dom_priority'] = merged_df1.apply(assign_dom_priority, axis=1)
merged_df1.sort_values(by=['dom_priority', 'rank'], inplace=True)
merged_df1['V_Rank'] = merged_df1.groupby(['streamId']).cumcount() + 1


merged_df3 = merged_df[(merged_df['streamId'] != '01') & (merged_df['streamId'] != 1)]
merged_df3['dom_priority'] = merged_df3.apply(assign_dom_priority, axis=1)
merged_df3.sort_values(by=['dom_priority','rank'], inplace=True)
merged_df3['V_Rank'] = merged_df3.groupby(['streamId']).cumcount() + 1

# roll_no_choices = {}
for index, row in merged_df1.iterrows():
    roll_no = row['rollNo']
    if roll_no in candidates.keys():
        candidates[roll_no].rank01 = row['V_Rank']

for index, row in merged_df3.iterrows():
    roll_no = row['rollNo']
    if roll_no in candidates.keys():
        candidates[roll_no].rank03 = row['V_Rank']        


In [91]:
info_df = pd.read_csv(candidate_profile_path)
info_df = info_df[(info_df['domicileId']==1) | (info_df['domicileId']=='1') | (info_df['domicileId']=='01')]
info_df = info_df.rename(columns={'rollno': 'rollNo'})
info_df = pd.merge(candidates_df, info_df, on='rollNo')
marks_df = pd.read_csv(quali_marks_path)
merged_df = pd.merge(marks_df, info_df, on='rollNo')


merged_df['DOB'] = pd.to_datetime(merged_df['DOB'], format='%d-%m-%Y')
merged_df['dom_priority'] = merged_df.apply(assign_dom_priority, axis=1)
# merged_df['rank'] = merged_df.sort_values(by=['dom_priority', 'percentageMarks', 'DOB'], ascending=[True, False, False]).groupby('roundNo_x').cumcount() + 1
merged_df['rank'] = merged_df.sort_values(by=['dom_priority', 'percentageMarks', 'DOB', 'rollNo'], ascending=[True, False, False, True]).groupby('boardId_x').cumcount() + 1
ranks_dict = dict(zip(merged_df['rollNo'], merged_df['rank']))

for rollNo in ranks_dict.keys():
    if rollNo in candidates.keys():
        candidates[rollNo].boardRank=ranks_dict[rollNo]

In [92]:
# Function to print candidate details
def print_all_candidates(candidates):
    for roll_no, candidate in candidates.items():
        print(f"Roll No: {candidate.roll_no}")
        print(f"  Board ID: {candidate.board_id}")
        print(f"  State ID: {candidate.state_id}")
        print(f"  Domicile ID: {candidate.domicile_id}")
        print(f"  Gender ID: {candidate.gender_id}")
        print(f"  Category ID: {candidate.category_id}")
        print(f"  Category Certificate No: {candidate.category_certificate_no}")
        print(f"  Choice List: {candidate.max_index}")
        print(f"  Rank 01: {candidate.rank01}")
        print(f"  Rank 03: {candidate.rank03}")
        print(f"  Board Rank : {candidate.boardRank}")
        print()


# Print all candidates' details
# print_all_candidates(candidates)

In [93]:
def spl_printer(roll_no):
    candidate=candidates[roll_no]
    print(f"Roll No: {candidate.roll_no}")
    print(f"  Board ID: {candidate.board_id}")
    print(f"  State ID: {candidate.state_id}")
    print(f"  Domicile ID: {candidate.domicile_id}")
    print(f"  Gender ID: {candidate.gender_id}")
    print(f"  Category ID: {candidate.category_id}")
    print(f"  Category Certificate No: {candidate.category_certificate_no}")
    print(f"  Choices: {candidate.max_index}")
    print(f"  Current Index: {candidate.current_index}")
    print(f"  Choice List:")
    for i in range(0, candidate.max_index):
        print(f"    {i+1}) {candidate.choice_list_mapper[i]} {candidate.choice_list[i].institute},{candidate.choice_list[i].program},{candidate.choice_list[i].category},{candidate.choice_list[i].subcategory} {candidate.choice_list[i].stream}")
    print(f"  Rank 01: {candidate.rank01}")
    print(f"  Rank 03: {candidate.rank03}")
    print(f"  Board Rank : {candidate.boardRank}")
    print()

In [94]:
def getRank(roll_no, program):
    if program.stream==3:
        return candidates[roll_no].rank03
    if program.stream==1:
        return candidates[roll_no].rank01
    elif program.stream==4: 
        return candidates[roll_no].boardRank

In [ ]:
def prioritizer():
    for roll in candidates.keys():
        candidate=candidates[roll]
        if candidate.rank01:
            if candidate.category_id==2:
                candidate.rank01+=10000000
        if candidate.rank03:
            if candidate.category_id==2:
                candidate.rank03+=10000000
        if candidate.boardRank:
            if candidate.category_id==2:
                candidate.boardRank+=10000000
                print(4)

    for roll in candidates.keys():    
        candidate3 = [c for c in candidates if candidates[c].rank03]
        candidate3.sort(key=lambda c: candidates[c].rank03)
        for i, c in enumerate(candidate3, start=1):
            candidates[c].rank03 = i
        candidate1 = [c for c in candidates if candidates[c].rank01]
        candidate1.sort(key=lambda c: candidates[c].rank01)
        for i, c in enumerate(candidate1, start=1):
            candidates[c].rank01 = i
        candidate4 = [c for c in candidates if candidates[c].boardRank]
        candidate4.sort(key=lambda c: candidates[c].boardRank)
        for i, c in enumerate(candidate4, start=1):
            candidates[c].boardRank = i
            
prioritizer()


In [97]:
def getLastRank(program):
    if len(program.waitlist)<program.capacity or len(program.waitlist)==0:
        return -1
    cand=heapq.heappop(program.waitlist)
    rk=-cand[0]
    heapq.heappush(program.waitlist, cand)
    return rk

In [98]:
def remove_candidate_with_roll_number(waitlist, roll_number):
    candidate_index = next((i for i, candidate in enumerate(waitlist) if candidate[1] == roll_number), None)
    if candidate_index is None:
        return None
    candidate = waitlist[candidate_index]
    waitlist[candidate_index] = waitlist[-1]
    waitlist.pop()
    if candidate_index < len(waitlist):
        heapq._siftup(waitlist, candidate_index)
        heapq._siftdown(waitlist, 0, candidate_index)
    
    return candidate

In [99]:
def checkAndAllocate(candidate, i, stream):
    if i<=candidate.max_index:
        preference = candidates[candidate.roll_no].choice_list[i-1]
        if preference and preference.stream == stream :
            if (preference.institute, preference.program, '01', 0, 'NO', 'B') not in iprograms.keys():  
                return
            iprogram = iprograms[preference.institute, preference.program, '01', 0, 'NO', 'B']
            if (0, preference.program, '01', preference.category, preference.subcategory, 'B') not in programs.keys():         
                return
            program = programs[0, preference.program, '01', preference.category, preference.subcategory, 'B']
            rk=getRank(roll_no, iprogram)
            if stream == 1:
                rk = candidate.rank01
            elif stream == 3:
                rk = candidate.rank03
            elif stream == 4:
                rk = candidate.boardRank
            else:
                rk = None  # or handle the default case as needed
            if rk and rk>0:
                if program.capacity>len(program.waitlist) and iprogram.capacity>len(iprogram.waitlist):
                    program.add_candidate(rk, roll_no)
                    iprogram.add_candidate(rk, roll_no)
                    candidate.current_index = i
                    candidate.cAlloc = preference
                    

In [100]:
def allot3():
    candidate3 = [c for c in candidates if candidates[c].rank03]
    candidate3.sort(key=lambda c: candidates[c].rank03)
    for roll_no in candidate3:
        candidate=candidates[roll_no]                  
        tempAlloc=candidate.cAlloc
        candidate.cAlloc=None
        if tempAlloc:
            remove_candidate_with_roll_number(iprograms[tempAlloc.institute, tempAlloc.program, '01', 0, 'NO', 'B'].waitlist, roll_no)
            remove_candidate_with_roll_number(programs[0, tempAlloc.program, '01', tempAlloc.category, tempAlloc.subcategory, 'B'].waitlist, roll_no)
        for i in range(1, candidate.current_index):   
            checkAndAllocate(candidate, i, 3)                   
            if candidate.cAlloc:
                break                         
        if candidate.cAlloc:
            continue    
        if tempAlloc:
            programs[0, tempAlloc.program, '01', tempAlloc.category, tempAlloc.subcategory, 'B'].add_candidate(getRank(roll_no, tempAlloc), roll_no)
            iprograms[tempAlloc.institute, tempAlloc.program, '01', 0, 'NO', 'B'].add_candidate(getRank(roll_no, tempAlloc), roll_no)
            candidate.cAlloc=tempAlloc
            continue
        for i in range(candidate.current_index, candidate.max_index+1): 
            checkAndAllocate(candidate, i, 3)                    
            if candidate.cAlloc:
                break       

allot3()

In [101]:
def allot1():
    candidate1 = [c for c in candidates if candidates[c].rank01]
    candidate1.sort(key=lambda c: candidates[c].rank01)
    for roll_no in candidate1:
        candidate=candidates[roll_no]                          
        tempAlloc=candidate.cAlloc
        candidate.cAlloc=None
        sum=0
        if tempAlloc:
            remove_candidate_with_roll_number(iprograms[tempAlloc.institute, tempAlloc.program, '01', 0, 'NO', 'B'].waitlist, roll_no)
            remove_candidate_with_roll_number(programs[0, tempAlloc.program, '01', tempAlloc.category, tempAlloc.subcategory, 'B'].waitlist, roll_no)
        for i in range(1, candidate.current_index):
            checkAndAllocate(candidate, i, 1)                   
            if candidate.cAlloc:
                break
        if candidate.cAlloc:
            continue
        if tempAlloc:
            programs[0, tempAlloc.program, '01', tempAlloc.category, tempAlloc.subcategory, 'B'].add_candidate(getRank(roll_no, tempAlloc), roll_no)
            iprograms[tempAlloc.institute, tempAlloc.program, '01', 0, 'NO', 'B'].add_candidate(getRank(roll_no, tempAlloc), roll_no)
            candidate.cAlloc=tempAlloc
            continue
        for i in range(candidate.current_index, candidate.max_index+1): 
            checkAndAllocate(candidate, i, 1)                 
            if candidate.cAlloc:
                break          
    
allot1()

In [102]:
def allot4():
    candidate4 = [c for c in candidates if candidates[c].boardRank]
    candidate4.sort(key=lambda c: candidates[c].boardRank)
    for roll_no in candidate4:
        candidate=candidates[roll_no]                          
        tempAlloc=candidate.cAlloc
        candidate.cAlloc=None
        if tempAlloc:
            remove_candidate_with_roll_number(iprograms[tempAlloc.institute, tempAlloc.program, '01', 0, 'NO', 'B'].waitlist, roll_no)
            remove_candidate_with_roll_number(programs[0, tempAlloc.program, '01', tempAlloc.category, tempAlloc.subcategory, 'B'].waitlist, roll_no)
        for i in range(1, candidate.current_index):  
            checkAndAllocate(candidate, i, 4)                       
            if candidate.cAlloc:
                break                     
        if candidate.cAlloc:
            continue    
        if tempAlloc:
            programs[0, tempAlloc.program, '01', tempAlloc.category, tempAlloc.subcategory, 'B'].add_candidate(getRank(roll_no, tempAlloc), roll_no)
            iprograms[tempAlloc.institute, tempAlloc.program, '01', 0, 'NO', 'B'].add_candidate(getRank(roll_no, tempAlloc), roll_no)
            candidate.cAlloc=tempAlloc
            continue
        for i in range(candidate.current_index, candidate.max_index+1):
            checkAndAllocate(candidate, i, 4)                      
            if candidate.cAlloc:
                break     

allot4()

In [103]:
def clearer():
    for program_key, program in programs.items():
        programs[program_key].waitlist= [candidate for candidate in programs[program_key].waitlist if candidates[candidate[1]].cAlloc and candidates[candidate[1]].cAlloc.category==program_key[3]  and candidates[candidate[1]].cAlloc.subcategory==program_key[4] and candidates[candidate[1]].cAlloc.program==program_key[1]]
    for program_key, program in iprograms.items():
        iprograms[program_key].waitlist= [candidate for candidate in iprograms[program_key].waitlist if candidates[candidate[1]].cAlloc and candidates[candidate[1]].cAlloc.institute==program_key[0] and candidates[candidate[1]].cAlloc.program==program_key[1]]

In [104]:
for i in range(0, len(candidates)):
    allot3()
    clearer()
    allot1()
    clearer()
    allot4()
    clearer()

In [ ]:
al=0
print("Final Allocations:")
for roll_no, candidate in candidates.items():
    if candidate.cAlloc:
        al+=1
        print(f"Candidate {roll_no} is allocated to program  Choice - {candidate.current_index}")
print(f"ALLOCATED SEATS:{al}")
sum=0
print("\nFinal Waitlists:")
for program_key, program in programs.items():
    sum=sum+len(program.waitlist)
    if len(program.waitlist)>0:
        print(f"Program {program_key} with {program.capacity} waitlist: {program.waitlist} with {program.capacity - len(program.waitlist)} remaining seats")

    

In [ ]:
rem_sum=0
for program_key, program in iprograms.items():
    rem_sum=rem_sum+program.capacity-len(program.waitlist)
    if program.capacity-len(program.waitlist)>0:
        print(f"Program {program_key} with {program.capacity} has {program.capacity - len(program.waitlist)} remaining seats. Stream : {program.stream}")

In [ ]:
rem_sum=0
for program_key, program in programs.items():
    rem_sum=rem_sum+program.capacity-len(program.waitlist)
    if program.capacity-len(program.waitlist)>0:
        print(f"Program {program_key} with {program.capacity} has {program.capacity - len(program.waitlist)} remaining seats. Stream : {program.stream}")
    else:
        print(f"Program {program_key} with {program.capacity} has no {program.capacity - len(program.waitlist)} remaining seats. Stream : {program.stream}")

        

In [ ]:
import csv

unallocated = []

with open('AN_Final_allocationsLinear.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Roll No", "Program Code", "Branch Code", "Quota", "Category", "Subcategory", "Gender", "Rank", "ChoiceIndex", "VirtualIndex"])
    for roll_no, candidate in candidates.items():
        if not candidate.cAlloc:
            unallocated.append(candidate.roll_no)
        else:
            ch=candidate.current_index-1
            if ch not in candidate.choice_list_mapper.keys():
                ch=candidate.max_index
            else:
                ch=candidate.choice_list_mapper[ch]
            writer.writerow([
                roll_no,
                candidate.cAlloc.institute,
                candidate.cAlloc.program,
                candidate.cAlloc.quota,
                candidate.cAlloc.category,
                candidate.cAlloc.subcategory,
                candidate.cAlloc.gender,
                getRank(roll_no, candidate.cAlloc),
                ch,
                candidate.current_index-1
            ])

print("Allocations generated")

# with open('unallocated_candidates.csv', mode='w', newline='') as file:
with open('final_unallocated_candidatesLinear.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Roll No"])
    for roll_no in unallocated:
        writer.writerow([roll_no])

print("Unallocations generated")

In [110]:
def allotReader(all_df):
    for index, row in all_df.iterrows():
        if row['allottedQuota'][:2]!='01':
               continue
        candidate=candidates[row['rollNo']]
        done.append(row['rollNo'])
        if candidate.cAlloc:
                if candidate.choice_list_mapper[candidate.current_index-1]!=row['optionNo']:
                        if candidate.cAlloc.institute != row['instituteId'] or candidate.cAlloc.program != row['programId'] or candidate.cAlloc.quota != row['allottedQuota'] or candidate.cAlloc.category != row['allottedCat'][:2] or candidate.cAlloc.subcategory.upper() != row['allottedCat'][2:].upper():
                                print(f'{candidate.roll_no} got ({candidate.choice_list_mapper[candidate.current_index-1]})({candidate.cAlloc.institute},{candidate.cAlloc.program},{candidate.cAlloc.quota},{candidate.cAlloc.category},{candidate.cAlloc.subcategory.upper()}) but ({row['optionNo']})({row['instituteId']},{row['programId']},{row['allottedQuota']},{row['allottedCat'][:2]},{row['allottedCat'][2:]}) in NIC {getRank(candidate.roll_no, candidate.cAlloc)} and {row['rank']}') 

        else:
                print(f"{candidate.roll_no} didn't get in ours but got ({row['optionNo']})({row['instituteId']},{row['programId']},{row['allottedQuota']},{row['allottedCat'][:2]},{row['allottedCat'][2:]}) in NIC's {row['rank']}")

all_df = pd.read_csv(allotment_path)
all_df = all_df[all_df['roundNo']==1]
done = []
allotReader(all_df)
for roll in candidates.keys():
       if not roll in done and candidates[roll].cAlloc:
              candidate=candidates[roll]
              print(f'{candidate.roll_no} got ({candidate.choice_list_mapper[candidate.current_index-1]})({candidate.cAlloc.institute},{candidate.cAlloc.program},{candidate.cAlloc.quota},{candidate.cAlloc.category},{candidate.cAlloc.subcategory.upper()}) but not in NIC {getRank(candidate.roll_no, candidate.cAlloc)}') 